In [2]:
# import lmdb
import pickle, re
from functools import partial
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from rdkit import Chem, RDLogger
from rdkit.Chem import Draw
RDLogger.DisableLog('rdApp.*')
from collections import Counter
from tqdm import tqdm
import multiprocessing as mp
import matplotlib.pyplot as plt 
import os
from typing import Iterable
from copy import deepcopy


In [15]:
path = '/home/lyt/projects/spec2mol/datasets/raman2mol/eval.pkl'
p = pd.read_pickle(path)
a=p['smiles'].values[0]
a

array([ 1,  6,  5, 19,  5,  5,  5, 15,  2, 16,  5,  5, 19,  4,  5, 19,  5,
        5, 15,  2, 16,  5,  5,  5, 19,  6, 23])

In [16]:
np.hstack([np.array([1]),a[1:-1]+1, np.array([2])])

array([ 1,  7,  6, 20,  6,  6,  6, 16,  3, 17,  6,  6, 20,  5,  6, 20,  6,
        6, 16,  3, 17,  6,  6,  6, 20,  7,  2])

In [17]:
def f(x):
    return np.hstack([np.array([1]),x[1:-1]+1, np.array([2])])

In [18]:
p['label']=p['label'].apply(f)
p.to_pickle(path)

### Random dataset

In [5]:
# import torch
# import random

# x = torch.randn(1000,1,1024)
# # 定义词汇表大小和 PAD 索引
# vocab_size = 19
# pad_idx = 0

# # 定义批次大小和最大序列长度
# batch_size = 1000
# max_seq_len = 10

# # 随机生成每个序列的长度
# seq_lengths = [random.randint(1, max_seq_len) for _ in range(batch_size)]

# # 生成随机序列
# sequences = []
# for length in seq_lengths:
#     seq = [random.randint(1, vocab_size - 1) for _ in range(length)]
#     sequences.append(torch.tensor(seq, dtype=torch.long))
# y = sequences
# y = [torch.hstack((torch.tensor([0]),i,torch.tensor([vocab_size+1]))) for i in y]
# dfs = pd.DataFrame({
#     'spectrum':[i for i in x],
#     'label':y
# })

### IR

In [ ]:
with open('/home/lyt/projects/sub_id-main/datasets/ir/polymerir.txt', 'r') as f:
    polymerfiles = [i.strip() for i in f.readlines()]
polymerfiles

In [2]:
alkane = '[CX4;H3,H2]'
alkene = '[CX3]=[CX3]'
alkyne = '[CX2]#[CX2]'
arene = '[cX3]1[cX3][cX3][cX3][cX3][cX3]1'
haloalkane = '[#6][F,Cl,Br,I]'
alcohol = '[#6][OX2H]'
aldehyde = '[CX3H1](=O)[#6,H]'
ketone = '[#6][CX3](=O)[#6]'
carboxylic_acid = '[CX3](=O)[OX2H]'
acid_anhydride = '[CX3](=[OX1])[OX2][CX3](=[OX1])'
acyl_halide = '[CX3](=[OX1])[F,Cl,Br,I]'
ester = '[#6][CX3](=O)[OX2H0][#6]'
ether = '[OD2]([#6])[#6]'
amine = '[NX3;H2,H1,H0;!$(NC=O)]'
amide = '[NX3][CX3](=[OX1])[#6]'
nitrile = '[NX1]#[CX2]'
imide = '[CX3](=[OX1])[NX3][CX3](=[OX1])'
imine = '[$([CX3]([#6])[#6]),$([CX3H][#6])]=[$([NX2][#6]),$([NX2H])]'
azo_compound = '[#6][NX2]=[NX2][#6]'
thiol = '[#16X2H]'
thial = '[CX3H1](=O)[#6,H]'
sulfone = '[#16X4](=[OX1])(=[OX1])([#6])[#6]'
sulfonic_acid = '[#16X4](=[OX1])(=[OX1])([#6])[OX2H]'
enol = '[OX2H][#6X3]=[#6]'
phenol = '[OX2H][cX3]:[c]'
hydrazine = '[NX3][NX3]'
enamine = '[NX3][CX3]=[CX3]'
isocyanate = '[NX2]=[C]=[O]'
isothiocyanate = '[NX2]=[C]=[S]'
phosphine = '[PX3]'
sulfonamide = '[#16X4]([NX3])(=[OX1])(=[OX1])[#6]'
sulfonate = '[#16X4](=[OX1])(=[OX1])([#6])[OX2H0]'
sulfoxide = '[#16X3]=[OX1]'
thioamide = '[NX3][CX3]=[SX1]'
hydrazone = '[NX3][NX2]=[#6]'
carbamate = '[NX3][CX3](=[OX1])[OX2H0]'
sulfide = '[#16X2H0]'

fg_list_extended = [
        alkane,
        alkene,
        alkyne,
        arene, 
        haloalkane, 
        alcohol,
        aldehyde,
        ketone, 
        carboxylic_acid, 
        acid_anhydride, 
        acyl_halide, 
        ester, 
        ether, 
        amine, 
        amide, 
        nitrile, 
        imide, 
        imine, 
        azo_compound,
        thiol, 
        thial, 
        sulfone, 
        sulfonic_acid, 
        enol, 
        phenol,
        hydrazine, 
        enamine, 
        isocyanate, 
        isothiocyanate,
        phosphine, 
        sulfonamide, 
        sulfonate, 
        sulfoxide, 
        thioamide, 
        hydrazone, 
        carbamate, 
        sulfide]
# elements = ['C','H','c','O','N', 'n','F','Cl','Br','I','S','P','#','=','(',')','@', '1', '2', ]

In [ ]:
# len(elements)

## 1. Washing and Labeling

In [87]:
root = '/home/lyt/projects/datasets/Raman'
def readspec(fname, ds):
    path = os.path.join(root,fname)
    # try:
    with open(path, "r", encoding="utf-8") as f:
        l = f.readlines()
    l = np.array([s.split()[0].split(',')[1] for s in l], dtype=np.float32)
    return np.nan_to_num(l)
    # except FileNotFoundError:
    #     print(path)
def mfi(inchi):
    try:
        return Chem.MolFromInchi(inchi)
    except: return None
def m2s(mol):
    try:
        return Chem.MolToSmiles(mol) 
    except: None
def interpolate(arr):
    return interp1d(np.linspace(400,4000,arr.shape[-1]),arr,kind='slinear')(np.linspace(400,4000,1024))
def label_subs(mol):
    return np.array([mol.HasSubstructMatch(Chem.MolFromSmarts(fg)) for fg in fg_list_extended], dtype=int)
def smiles_tokenizer(smiles):
    pattern = r"(\[[^\]]+\]|Cl|Br|[A-Za-z0-9=@#%\*\-\+\(\)\.\/\$\\])"
    tokens = re.findall(pattern, smiles)
    return tokens
def label_token(s: str, lst: list[str]) -> str:
    label = [0] + [lst.index(i)+1 for i in smiles_tokenizer(s)]
    label.append(len(lst)+1)
    return np.array(label, dtype=int)
def count_element(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return list(set([atom.GetSymbol() for atom in mol.GetAtoms()]))

def count_token(text: str, subs: Iterable[str]) -> str:
    try:
        if not subs:
            return text
        pattern = '|'.join(map(re.escape, subs))
        return re.sub(pattern, '', text)
    except: return None

In [95]:
df_label = []
dirlist = os.listdir(root) #polymerfiles  
for d in tqdm(dirlist, total=len(dirlist), ncols=100):
    try:
        inchifile = root+'/'+d+'/inchi.txt'
        with open(inchifile, 'r') as f:
            inchis = f.readlines()
        try: 
            ds = d +'/data'
            specfiles = sorted(os.listdir(root+'/'+ds), key=lambda name: int(name.split('.')[0]))
        except: 
            ds = d +'/result/data'
            specfiles = sorted(os.listdir(root+'/'+ds), key=lambda name: int(name.split('.')[0]))
        specfiles = [os.path.join(ds,f) for f in specfiles]
        if len(inchis)!=len(specfiles):
            inchis = inchis[:len(specfiles)]
        df = pd.DataFrame({
            "filename": specfiles,
            "inchi": inchis
        })
        df_label.append(df)
    except Exception as e:
        print(f'{d}: {e}')
df_label = pd.concat([d for d in df_label if not d.empty], ignore_index=True)

100%|██████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 122.42it/s]

RZ3X: 'utf-8' codec can't decode byte 0xa6 in position 7723: invalid start byte


,filename,inchi
0,QR2X/result/data/0.txt,"InChI=1S/C8H4F13N3/c9-3(10,1-2-23-24-22)4(11,1..."
1,QR2X/result/data/1.txt,"InChI=1S/C28H60O13Si/c1-5-39-42(40-6-2,41-7-3)..."
2,QR2X/result/data/2.txt,"InChI=1S/C38H77ClO16SSi/c1-4-53-57(54-5-2,55-6..."
3,QR2X/result/data/3.txt,"InChI=1S/C5H11O4P/c1-7-10(6,8-2)4-5-3-9-5/h5H,..."
4,QR2X/result/data/4.txt,MMA co TMI\n
...,...,...
244,QR2X/result/data/244.txt,"InChI=1S/C12H7F13O4/c13-7(14,3-4-29-6(28)2-1-5..."
245,QR2X/result/data/245.txt,Polystyrene sulfonate\n
246,QR2X/result/data/246.txt,Copolymer [styrene-co-phosphonic diacid monome...
247,QR2X/result/data/247.txt,Poly(sodium 4-styrenesulfonate)\n


In [96]:
# df_label.drop(df_label[~df_label['inchi'].str.startswith('InChI')].index, inplace=True) # no "~" for unlabeled spectra
df_label["spectrum"] = df_label["filename"].apply(partial(readspec, ds=ds))

''' no need for unlabeled spectra'''
df_label["mol"] = df_label["inchi"].apply(mfi)
# df_label = df_label[df_label["mol"].apply(lambda x: isinstance(x, Chem.rdchem.Mol))].reset_index(drop=True) 

df_label['smiles'] = df_label['mol'].apply(m2s) 
# dfs['label'] = dfs['mol'].apply(labeling)  
# dfs['label'] = dfs['smiles'].apply(labeling)  
df_label = df_label.drop(['filename', 'inchi', 'mol'], axis=1) 
'''no need for unlabeled spectra'''

df_label['spectrum'] = df_label['spectrum'].apply(interpolate)
df_label

,spectrum,smiles
0,"[0.02040800079703331, 0.02040800079703331, 0.0...",COc1ccc(C(Cl)(c2ccccc2)c2ccc(OC)cc2)cc1
1,"[0.024490000680088997, 0.028571000322699543, 0...",N=c1nc(O)c2ncc(O)nc2[nH]1
2,"[0.012245000340044498, 0.014286000281572342, 0...",N=c1nc(O)c2nc(C(=O)O)cnc2[nH]1
3,"[0.008163000456988811, 0.008163000456988811, 0...",Fc1cc(F)c(F)c(S)c1F
4,"[0.008163000456988811, 0.008163000456988811, 0...",Nc1cccc(O)c1
...,...,...
22484,"[0.0, 0.0, 0.0, 0.0066548801379557095, 0.00834...",O=C(O)/C=C/C(=O)OCCC(F)(F)C(F)(F)C(F)(F)C(F)(F...
22485,"[0.0, 0.0, 0.0, 0.011646651255944645, 0.014286...",None
22486,"[0.0, 0.0, 0.0, 0.004990956052329375, 0.006122...",None
22487,"[0.0, 0.0, 0.0, 0.009982727549948332, 0.012245...",None


In [28]:
df_unlabel = deepcopy(df_label)

In [29]:
e_all = []
t_all = []
for i in df_label['smiles'].values:
    if i: e_all+=count_element(i) 
Counter(e_all)

Counter({'C': 19571,
         'O': 16137,
         'N': 9607,
         'Cl': 3635,
         'S': 2423,
         'F': 2275,
         'Br': 1659,
         'Na': 594,
         'P': 560,
         'B': 482,
         'I': 448,
         'Si': 324,
         'K': 236,
         'Sn': 149,
         'Ca': 90,
         'H': 79,
         'Cr': 74,
         'Pb': 72,
         'Co': 70,
         'Al': 69,
         'Ba': 65,
         'Li': 63,
         'Zn': 62,
         'Hg': 58,
         'Mo': 57,
         'Fe': 54,
         'Ti': 52,
         'As': 51,
         'Mg': 49,
         'Ce': 46,
         'W': 45,
         'Se': 43,
         'Cs': 43,
         'Sb': 40,
         'Bi': 31,
         'Ag': 30,
         'Pd': 29,
         'Cu': 27,
         'Pt': 27,
         'Zr': 26,
         'Mn': 24,
         'Cd': 24,
         'V': 23,
         'Ni': 23,
         'Sr': 21,
         'Eu': 20,
         'La': 20,
         'In': 20,
         'Te': 19,
         'Th': 19,
         'Rh': 18,
         'Ge': 15,
 

In [31]:
for i in tqdm(df_label['smiles'].values, total=df_label.index.to_numpy().max(), ncols=100):
    if i:
        t_all+=count_token(i, e_all)
Counter(t_all)

  0%|                                                                     | 0/22488 [00:00<?, ?it/s]

22489it [05:58, 62.80it/s]                                                                          


Counter({'c': 109733,
         '(': 51270,
         ')': 51270,
         '1': 36864,
         '[': 28964,
         ']': 28964,
         '=': 24493,
         '2': 16346,
         '.': 13955,
         '@': 13311,
         '-': 9536,
         '+': 5822,
         'n': 5186,
         '3': 4274,
         '/': 1738,
         '4': 1726,
         '#': 1538,
         'a': 1168,
         'o': 810,
         'i': 552,
         's': 478,
         '\\': 293,
         '5': 249,
         'e': 175,
         'b': 174,
         'r': 142,
         '6': 72,
         'd': 62,
         '7': 36,
         '8': 30,
         'u': 28,
         't': 27,
         '9': 18,
         'm': 11})

In [10]:
tokens = ['Cl','Br','C','c','O','o','N','n','S','s','F','I','P','(',')','=','#','1','2','3','@']
len(tokens)

21

In [97]:
df_label['label'] = df_label['smiles'].apply(partial(count_token, subs=tokens))
df_label

,spectrum,smiles,label
0,"[0.02040800079703331, 0.02040800079703331, 0.0...",COc1ccc(C(Cl)(c2ccccc2)c2ccc(OC)cc2)cc1,
1,"[0.024490000680088997, 0.028571000322699543, 0...",N=c1nc(O)c2ncc(O)nc2[nH]1,[H]
2,"[0.012245000340044498, 0.014286000281572342, 0...",N=c1nc(O)c2nc(C(=O)O)cnc2[nH]1,[H]
3,"[0.008163000456988811, 0.008163000456988811, 0...",Fc1cc(F)c(F)c(S)c1F,
4,"[0.008163000456988811, 0.008163000456988811, 0...",Nc1cccc(O)c1,
...,...,...,...
22484,"[0.0, 0.0, 0.0, 0.0066548801379557095, 0.00834...",O=C(O)/C=C/C(=O)OCCC(F)(F)C(F)(F)C(F)(F)C(F)(F...,//
22485,"[0.0, 0.0, 0.0, 0.011646651255944645, 0.014286...",None,None
22486,"[0.0, 0.0, 0.0, 0.004990956052329375, 0.006122...",None,None
22487,"[0.0, 0.0, 0.0, 0.009982727549948332, 0.012245...",None,None


In [98]:
df_label = df_label.drop(df_label[~(df_label['label']=='')].index)
df_label

,spectrum,smiles,label
0,"[0.02040800079703331, 0.02040800079703331, 0.0...",COc1ccc(C(Cl)(c2ccccc2)c2ccc(OC)cc2)cc1,
3,"[0.008163000456988811, 0.008163000456988811, 0...",Fc1cc(F)c(F)c(S)c1F,
4,"[0.008163000456988811, 0.008163000456988811, 0...",Nc1cccc(O)c1,
5,"[0.010204000398516655, 0.008163000456988811, 0...",C1C2CC3CC1CC(C2)C3,
6,"[0.008163000456988811, 0.008871265343833763, 0...",OCc1c(Cl)cccc1Cl,
...,...,...,...
22451,"[0.0, 0.0, 0.0, 0.026620334478592328, 0.032653...",C=CCOCCCCCCCCCCCCOCC=C,
22452,"[0.0, 0.0, 0.0, 0.0066548801379557095, 0.00816...",CN1C(=O)C=CC1=O,
22453,"[0.0, 0.0, 0.0, 0.014973683222647685, 0.018189...",OCCn1ccnc1,
22454,"[0.0, 0.0, 0.0, 0.009982727549948332, 0.012245...",CC(C)(C)c1cc(Cc2cc(C(C)(C)C)c(O)c(C(C)(C)C)c2)...,


In [ ]:
df_label['label'] = df_label['smiles'].apply(partial(label_token, lst=tokens))
df_label

In [ ]:
print(df_unlabel.index.to_numpy().max())
df_unlabel['smiles'] = df_unlabel['inchi'].apply(mfi)
# for smiles in df_label['smiles'].values:
df_unlabel.drop(df_label.index)


In [ ]:
print(df_unlabel.index.to_numpy().max(), df_label.index.to_numpy().max())

In [49]:
type('')

str

## 2. Splitting

In [32]:
### tts
ds = 'raman2mol'
rng = np.random.default_rng(42)

idx = df_label.index.to_numpy()
rng.shuffle(idx)

n = len(idx)
train_end, val_end = int(0.85 * n), int(0.9 * n)

train_df = df_label.loc[idx[:train_end]]
val_df   = df_label.loc[idx[train_end:]]# no val_end]] for pretraining
test_df  = df_label.loc[idx[val_end:]]



In [ ]:
train_df.to_pickle(f'/home/lyt/projects/spec2mol/datasets/{ds}/train.pkl')
val_df.to_pickle(f'/home/lyt/projects/spec2mol/datasets/{ds}/eval.pkl')
test_df.to_pickle(f'/home/lyt/projects/spec2mol/datasets/{ds}/test.pkl')
idx.max()

In [ ]:
np.vstack(np.vstack(df['smiles'].values))

## 2. Creating with lmdb

In [16]:
db = lmdb.open('/data/YantiLiu/projects/substructure-ID/datasets/qm9s_raman/qm9s_raman_eval.lmdb', subdir=False, lock=False, map_size=int(1e11))

# Open a transaction and perform a read operation
with db.begin() as txn:
    dset = list(txn.cursor().iternext(values=False))

In [17]:
d1 = []
s1 = []
l1 = []
with db.begin() as txn:
    keys = list(txn.cursor().iternext(values=False))
    for i in keys:
        obj = pickle.loads(txn.get(i))
        d1.append(obj['spectra'])
        s1.append(obj['smiles'])
        l1.append(obj['substructures'])
d1, s1, l1 = np.vstack(d1), np.hstack(s1), np.vstack(l1)

In [54]:
dset = np.load('/data/YantiLiu/projects/multispec/dataset/raman_ir_qm9s.npz', allow_pickle=True)
d , s , l = dset['ir'], dset['smiles'], dset['label_957']

In [ ]:
smiles = np.load('/data/YantiLiu/projects/subs_id/datasets/qm9s_raman/eval_s.npy')
ids = np.hstack([np.where(s==e)[0] for e in tqdm(smiles, total=len(smiles))])


In [63]:
np.save('/data/YantiLiu/projects/subs_id/datasets/qm9s_ir/eval_y.npy', l[ids])
np.save('/data/YantiLiu/projects/subs_id/datasets/qm9s_ir/eval_x.npy', d[ids])
np.save('/data/YantiLiu/projects/subs_id/datasets/qm9s_ir/eval_s.npy', s[ids])

In [ ]:
(np.load('/data/YantiLiu/projects/subs_id/datasets/qm9s_raman/eval_s.npy') == np.load('/data/YantiLiu/projects/subs_id/datasets/qm9s_ir/eval_s.npy')
).all()

In [ ]:
i=1000
raman = np.load('/data/YantiLiu/projects/subs_id/datasets/qm9s_raman/test_x.npy')[i]
ir = np.load('/data/YantiLiu/projects/subs_id/datasets/qm9s_ir/test_x.npy')[i][::-1]
smiles = np.load('/data/YantiLiu/projects/subs_id/datasets/qm9s_ir/test_s.npy')[i]
plt.plot(raman)
plt.plot(ir)
plt.title(smiles)


In [ ]:
mol=Chem.MolFromSmiles(smiles)
Draw.MolsToGridImage(mol, size=(150,150), kekulize=True)